In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4
! pip install --ignore-installed -q spark-nlp==2.7.1

debconf: delaying package configuration, since apt-utils is not installed
openjdk version "1.8.0_282"
OpenJDK Runtime Environment (build 1.8.0_282-8u282-b08-0ubuntu1~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.282-b08, mixed mode)


In [3]:
import sparknlp

spark = sparknlp.start(gpu = True) # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)

from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd

print("Spark NLP version", sparknlp.version())

print("Apache Spark version:", spark.version)

spark

Spark NLP version 2.7.1
Apache Spark version: 2.4.4


## Read the training data 

In [4]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("/kaggle/input/nlp-getting-started/train.csv")

trainDataset.show(truncate=50)

+---+-------+--------+--------------------------------------------------+------+
| id|keyword|location|                                              text|target|
+---+-------+--------+--------------------------------------------------+------+
|  1|   null|    null|Our Deeds are the Reason of this #earthquake Ma...|     1|
|  4|   null|    null|            Forest fire near La Ronge Sask. Canada|     1|
|  5|   null|    null|All residents asked to 'shelter in place' are b...|     1|
|  6|   null|    null|13,000 people receive #wildfires evacuation ord...|     1|
|  7|   null|    null|Just got sent this photo from Ruby #Alaska as s...|     1|
|  8|   null|    null|#RockyFire Update => California Hwy. 20 closed ...|     1|
| 10|   null|    null|#flood #disaster Heavy rain causes flash floodi...|     1|
| 13|   null|    null|I'm on top of the hill and I can see a fire in ...|     1|
| 14|   null|    null|There's an emergency evacuation happening now i...|     1|
| 15|   null|    null|I'm af

In [5]:
trainDataset.count()

8387

## identify cases of missing target 

In [6]:
from pyspark.sql.functions import col

trainDataset.groupBy("target") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+------+-----+
|target|count|
+------+-----+
|     0| 4095|
|     1| 3081|
|  null| 1211|
+------+-----+



## check for text > 512 in length

In [7]:
from pyspark.sql.functions import length
trainDataset.where(length(col("text")) > 512).show()

+---+-------+--------+----+------+
| id|keyword|location|text|target|
+---+-------+--------+----+------+
+---+-------+--------+----+------+



## drop missing values from the text and target columns

In [8]:
train = trainDataset.dropna(subset=['text', 'target'])

In [9]:
train.groupBy("target") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+------+-----+
|target|count|
+------+-----+
|     0| 4095|
|     1| 3081|
+------+-----+



## preprocessing pipeline

In [10]:
document_assembler = DocumentAssembler() \
.setInputCol("text") \
.setOutputCol("document") \
.setCleanupMode("shrink")
    
tokenizer = Tokenizer() \
.setInputCols(["document"]) \
.setOutputCol("token") \
.setSplitChars(['-']) \
.setContextChars(['(', ')', '?', '!', '#', '@']) 

normalizer = Normalizer() \
.setInputCols(["token"]) \
.setOutputCol("normalized")\
.setCleanupPatterns(["[^\w\d\s]"]) 

stopwords_cleaner = StopWordsCleaner()\
.setInputCols("normalized")\
.setOutputCol("cleanTokens")\
.setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
.setInputCols(["cleanTokens"]) \
.setOutputCol("lemma")

preproc_pipeline = Pipeline(
  stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


## visualize results of the steps in pipeline

In [11]:
pipelineModel = preproc_pipeline.fit(train)
%time result = pipelineModel.transform(train).collect()

CPU times: user 3.63 s, sys: 216 ms, total: 3.85 s
Wall time: 18.6 s


In [12]:
from pyspark.sql import Row

text = "As she sat watching the world go by, something caught her eye. It wasn't so much its color or shape, but the way it was moving."
df = spark.createDataFrame(list(map(lambda x: Row(text=x), [text])), ["text"])
%time result = pipelineModel.transform(df).collect()

CPU times: user 154 ms, sys: 24.6 ms, total: 178 ms
Wall time: 1.32 s


In [13]:
empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = preproc_pipeline.fit(empty_df)


In [14]:
from sparknlp.base import LightPipeline

light_model = LightPipeline(pipelineModel)

%time light_result = light_model.annotate("As she sat watching the world go by, something caught her eye. It wasn't so much its color or shape, but the way it was moving.")

CPU times: user 15.3 ms, sys: 15.2 ms, total: 30.5 ms
Wall time: 60 ms


In [15]:
light_result.keys()

dict_keys(['lemma', 'document', 'normalized', 'cleanTokens', 'token'])

In [16]:
list(zip(light_result['token'], light_result['normalized'], light_result['cleanTokens'],  light_result['lemma']))

[('As', 'As', 'sat', 'sit'),
 ('she', 'she', 'watching', 'watch'),
 ('sat', 'sat', 'world', 'world'),
 ('watching', 'watching', 'go', 'go'),
 ('the', 'the', 'something', 'something'),
 ('world', 'world', 'caught', 'catch'),
 ('go', 'go', 'eye', 'eye'),
 ('by,', 'by', 'wasnt', 'wasnt'),
 ('something', 'something', 'much', 'much'),
 ('caught', 'caught', 'color', 'color'),
 ('her', 'her', 'shape', 'shape'),
 ('eye.', 'eye', 'way', 'way'),
 ('It', 'It', 'moving', 'move')]

## Classification using GloVe

In [17]:
document_assembler = DocumentAssembler() \
.setInputCol("text") \
.setOutputCol("document") \
.setCleanupMode("shrink")
    
tokenizer = Tokenizer() \
.setInputCols(["document"]) \
.setOutputCol("token") \
.setSplitChars(['-']) \
.setContextChars(['(', ')', '?', '!', '#', '@']) 

normalizer = Normalizer() \
.setInputCols(["token"]) \
.setOutputCol("normalized")\
.setCleanupPatterns(["[^\w\d\s]"]) 

stopwords_cleaner = StopWordsCleaner()\
.setInputCols("normalized")\
.setOutputCol("cleanTokens")\
.setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
.setInputCols(["cleanTokens"]) \
.setOutputCol("lemma")

glove_embeddings = WordEmbeddingsModel().pretrained() \
.setInputCols(["document",'lemma'])\
.setOutputCol("embeddings")\
.setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
.setInputCols(["document", "embeddings"]) \
.setOutputCol("sentence_embeddings") \
.setPoolingStrategy("AVERAGE")

classsifierdl = ClassifierDLApproach()\
.setInputCols(["sentence_embeddings"])\
.setOutputCol("class")\
.setLabelColumn("target")\
.setMaxEpochs(5)\
.setLr(0.001)\
.setBatchSize(8)\
.setEnableOutputLogs(True)
#.setOutputLogsPath('logs')

glove_clf_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            glove_embeddings,
            embeddingsSentence,
            classsifierdl])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


## split training data into train/validation sets

In [18]:
(train_df, val_df) = train.randomSplit([0.7, 0.3], seed = 8)
print("Training Dataset Count: " + str(train_df.count()))
print("Validation Dataset Count: " + str(val_df.count()))

Training Dataset Count: 5036
Validation Dataset Count: 2140


In [19]:
glove_clf_pipelineModel = glove_clf_pipeline.fit(train_df)

In [20]:
!cd ~/annotator_logs && ls -l

total 4
-rw-r--r-- 1 root root 449 Mar 16 09:22 ClassifierDLApproach_af6576cde5d2.log


In [21]:
!cat ~/annotator_logs/ClassifierDLApproach_b5fa4fd51592.log

cat: /root/annotator_logs/ClassifierDLApproach_b5fa4fd51592.log: No such file or directory


In [22]:
# get the predictions on validation Set

preds = glove_clf_pipelineModel.transform(val_df)

In [23]:
preds.select('text','target',"class.result").show(10, truncate=80)

+--------------------------------------------------------------------------------+------+------+
|                                                                            text|target|result|
+--------------------------------------------------------------------------------+------+------+
|http://t.co/GKYe6gjTk5 Had a #personalinjury accident this summer? Read our a...|     0|   [0]|
|Meet Brinco your own personal earthquake snd tsunami early warning beacon. ht...|     1|   [1]|
|                                                    I want some tsunami take out|     0|   [1]|
|Just stop fucking saying ÛÏa whole Û÷notherÛ. It just sounds fucking stup...|     0|   [0]|
|Brain twister homefolks are opinionated over against proposal modernized cana...|     0|   [0]|
|Crazy Mom Threw Teen Daughter a NUDE Twister Sex Party According To Her Frien...|     0|   [0]|
|     The Sharper Image Viper 24' Hardside Twister (Black) http://t.co/FXk3zsj2PE|     0|   [0]|
|                         Brai

In [24]:
preds_df = preds.select('text','target',"class.result").toPandas()

# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

In [25]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report(preds_df['result'], preds_df['target']))

              precision    recall  f1-score   support

           0       0.85      0.81      0.83      1276
           1       0.73      0.78      0.76       864

    accuracy                           0.80      2140
   macro avg       0.79      0.79      0.79      2140
weighted avg       0.80      0.80      0.80      2140



## Classification using Elmo


In [26]:
document_assembler = DocumentAssembler() \
.setInputCol("text") \
.setOutputCol("document") \
.setCleanupMode("shrink")
    
tokenizer = Tokenizer() \
.setInputCols(["document"]) \
.setOutputCol("token") \
.setSplitChars(['-']) \
.setContextChars(['(', ')', '?', '!', '#', '@']) 

normalizer = Normalizer() \
.setInputCols(["token"]) \
.setOutputCol("normalized")\
.setCleanupPatterns(["[^\w\d\s]"]) 

stopwords_cleaner = StopWordsCleaner()\
.setInputCols("normalized")\
.setOutputCol("cleanTokens")\
.setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
.setInputCols(["cleanTokens"]) \
.setOutputCol("lemma")
    
elmo_embeddings = ElmoEmbeddings.pretrained('elmo')\
.setInputCols(["document", "token"])\
.setOutputCol("embeddings")\
.setPoolingLayer('elmo')# default --> elmo

embeddingsSentence = SentenceEmbeddings() \
.setInputCols(["document", "embeddings"]) \
.setOutputCol("sentence_embeddings") \
.setPoolingStrategy("AVERAGE")

classsifierdl = ClassifierDLApproach()\
.setInputCols(["sentence_embeddings"])\
.setOutputCol("class")\
.setLabelColumn("target")\
.setMaxEpochs(5)\
.setLr(0.001)\
.setBatchSize(8)\
.setEnableOutputLogs(True)
#.setOutputLogsPath('logs')

elmo_clf_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner,
            lemma,
            elmo_embeddings,
            embeddingsSentence,
            classsifierdl])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
elmo download started this may take some time.
Approximate size to download 334.1 MB
[OK!]


In [27]:
elmo_clf_pipelineModel = elmo_clf_pipeline.fit(train_df)

In [28]:
!cd /root/annotator_logs && ls -lt

total 8
-rw-r--r-- 1 root root 444 Mar 16 09:59 ClassifierDLApproach_90842185ddd7.log
-rw-r--r-- 1 root root 449 Mar 16 09:22 ClassifierDLApproach_af6576cde5d2.log


In [29]:
!cat /root/annotator_logs/ClassifierDLApproach_72aceb5a2cde.log

cat: /root/annotator_logs/ClassifierDLApproach_72aceb5a2cde.log: No such file or directory


In [30]:
preds = elmo_clf_pipelineModel.transform(val_df)

In [31]:
preds.select('text','target',"class.result").show(10, truncate=80)

+--------------------------------------------------------------------------------+------+------+
|                                                                            text|target|result|
+--------------------------------------------------------------------------------+------+------+
|http://t.co/GKYe6gjTk5 Had a #personalinjury accident this summer? Read our a...|     0|   [0]|
|Meet Brinco your own personal earthquake snd tsunami early warning beacon. ht...|     1|   [1]|
|                                                    I want some tsunami take out|     0|   [0]|
|Just stop fucking saying ÛÏa whole Û÷notherÛ. It just sounds fucking stup...|     0|   [0]|
|Brain twister homefolks are opinionated over against proposal modernized cana...|     0|   [0]|
|Crazy Mom Threw Teen Daughter a NUDE Twister Sex Party According To Her Frien...|     0|   [0]|
|     The Sharper Image Viper 24' Hardside Twister (Black) http://t.co/FXk3zsj2PE|     0|   [0]|
|                         Brai

In [32]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report



preds_df = preds.select('text','target',"class.result").toPandas()

preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

print (classification_report(preds_df['result'], preds_df['target']))

              precision    recall  f1-score   support

           0       0.90      0.81      0.85      1362
           1       0.72      0.85      0.78       778

    accuracy                           0.82      2140
   macro avg       0.81      0.83      0.81      2140
weighted avg       0.84      0.82      0.83      2140



## Classification using BERT


In [33]:
document_assembler = DocumentAssembler() \
.setInputCol("text") \
.setOutputCol("document") \
.setCleanupMode("shrink")
    
tokenizer = Tokenizer() \
.setInputCols(["document"]) \
.setOutputCol("token") \
.setSplitChars(['-']) \
.setContextChars(['(', ')', '?', '!', '#', '@']) 

normalizer = Normalizer() \
.setInputCols(["token"]) \
.setOutputCol("normalized")\
.setCleanupPatterns(["[^\w\d\s]"]) 

stopwords_cleaner = StopWordsCleaner()\
.setInputCols("normalized")\
.setOutputCol("cleanTokens")\
.setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
.setInputCols(["cleanTokens"]) \
.setOutputCol("lemma")

bert_embeddings = BertEmbeddings().pretrained(name='bert_base_cased', lang='en') \
.setInputCols(["document",'token'])\
.setOutputCol("embeddings")

embeddingsSentence = SentenceEmbeddings() \
.setInputCols(["document", "embeddings"]) \
.setOutputCol("sentence_embeddings") \
.setPoolingStrategy("AVERAGE")

classsifierdl = ClassifierDLApproach()\
.setInputCols(["sentence_embeddings"])\
.setOutputCol("class")\
.setLabelColumn("target")\
.setMaxEpochs(5)\
.setLr(0.001)\
.setBatchSize(8) \
.setEnableOutputLogs(True)
#.setOutputLogsPath('logs')

bert_clf_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner,
            lemma,
            bert_embeddings,
            embeddingsSentence,
            classsifierdl])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]


In [34]:
bert_clf_pipelineModel = bert_clf_pipeline.fit(train_df)

In [35]:
!cd /root/annotator_logs && ls -lt

total 12
-rw-r--r-- 1 root root 443 Mar 16 10:25 ClassifierDLApproach_c60cf9d00056.log
-rw-r--r-- 1 root root 444 Mar 16 09:59 ClassifierDLApproach_90842185ddd7.log
-rw-r--r-- 1 root root 449 Mar 16 09:22 ClassifierDLApproach_af6576cde5d2.log


In [36]:
!cat /root/annotator_logs/ClassifierDLApproach_5571ae93049a.log

cat: /root/annotator_logs/ClassifierDLApproach_5571ae93049a.log: No such file or directory


In [37]:
preds = bert_clf_pipelineModel.transform(val_df)

In [38]:
preds.select('text','target',"class.result").show(10, truncate=80)

+--------------------------------------------------------------------------------+------+------+
|                                                                            text|target|result|
+--------------------------------------------------------------------------------+------+------+
|http://t.co/GKYe6gjTk5 Had a #personalinjury accident this summer? Read our a...|     0|   [0]|
|Meet Brinco your own personal earthquake snd tsunami early warning beacon. ht...|     1|   [0]|
|                                                    I want some tsunami take out|     0|   [0]|
|Just stop fucking saying ÛÏa whole Û÷notherÛ. It just sounds fucking stup...|     0|   [0]|
|Brain twister homefolks are opinionated over against proposal modernized cana...|     0|   [0]|
|Crazy Mom Threw Teen Daughter a NUDE Twister Sex Party According To Her Frien...|     0|   [0]|
|     The Sharper Image Viper 24' Hardside Twister (Black) http://t.co/FXk3zsj2PE|     0|   [0]|
|                         Brai

In [39]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report



preds_df = preds.select('text','target',"class.result").toPandas()

preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

print (classification_report(preds_df['result'], preds_df['target']))

              precision    recall  f1-score   support

           0       0.93      0.77      0.85      1466
           1       0.64      0.88      0.74       674

    accuracy                           0.81      2140
   macro avg       0.79      0.83      0.79      2140
weighted avg       0.84      0.81      0.81      2140

